## AlexNet

&emsp;&emsp;AlexNet是一个更深更大的LeNet,它主要改进的地方是在:

1. 丢弃法
2. ReLu
3. MaxPooling: 取的是最大值，使得输出的值会比较大，梯度相对来说会比较大，训练相对来说会容易一点。
4. 数据增强

In [1]:
import torch
import torch.nn as nn

net = nn.Sequential(
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2), nn.Flatten(),
    nn.Linear(6400, 4096), nn.ReLU(), nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(p=0.5),
    nn.Linear(4096, 10)
)

&emsp;&emsp;我们构造一个单通道数据，来观察每一层输出的形状:

In [2]:
X = torch.randn(size=(1, 1, 224, 224), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, "Out Put Shape: ", X.shape)

Conv2d Out Put Shape:  torch.Size([1, 96, 54, 54])
ReLU Out Put Shape:  torch.Size([1, 96, 54, 54])
MaxPool2d Out Put Shape:  torch.Size([1, 96, 26, 26])
Conv2d Out Put Shape:  torch.Size([1, 256, 26, 26])
ReLU Out Put Shape:  torch.Size([1, 256, 26, 26])
MaxPool2d Out Put Shape:  torch.Size([1, 256, 12, 12])
Conv2d Out Put Shape:  torch.Size([1, 384, 12, 12])
ReLU Out Put Shape:  torch.Size([1, 384, 12, 12])
Conv2d Out Put Shape:  torch.Size([1, 384, 12, 12])
ReLU Out Put Shape:  torch.Size([1, 384, 12, 12])
Conv2d Out Put Shape:  torch.Size([1, 256, 12, 12])
ReLU Out Put Shape:  torch.Size([1, 256, 12, 12])
MaxPool2d Out Put Shape:  torch.Size([1, 256, 5, 5])
Flatten Out Put Shape:  torch.Size([1, 6400])
Linear Out Put Shape:  torch.Size([1, 4096])
ReLU Out Put Shape:  torch.Size([1, 4096])
Dropout Out Put Shape:  torch.Size([1, 4096])
Linear Out Put Shape:  torch.Size([1, 4096])
ReLU Out Put Shape:  torch.Size([1, 4096])
Dropout Out Put Shape:  torch.Size([1, 4096])
Linear Out Put S

## 加载数据集

In [3]:
import torchvision
from torchvision import transforms
from torch.utils import data

def get_dataloader_workers():
    """Use 4 processes to read the data."""
    return 4

def load_data_fashion_mnist(batch_size, resize=None):
    """Download the Fashion-MNIST dataset and then load it into memory."""
    trans = [transforms.ToTensor()]
    
    if resize:
        trans.insert(0, transforms.Resize(resize))
        
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(root="../data", train=False, transform=trans, download=True)
    
    return (data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=get_dataloader_workers()),
            data.DataLoader(mnist_test, batch_size, shuffle=False, num_workers=get_dataloader_workers()))

&emsp;&emsp;`Fashion_Mnist`图像分辨率低于`ImageNet`图像，我们将其增加到$224 \times 224$:

In [4]:
train_iter, test_iter = load_data_fashion_mnist(batch_size=128, resize=224)

/Users/tinyzqh/miniforge3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-wdh4qigd/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
class Accumulator:
    """For accumulating sums over `n` variables."""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [6]:
argmax = lambda x, *args, **kwargs: x.argmax(*args, **kwargs)
reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
astype = lambda x, *args, **kwargs: x.type(*args, **kwargs)

def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = argmax(y_hat, axis=1)
    cmp = astype(y_hat, y.dtype) == y
    return float(reduce_sum(astype(cmp, y.dtype)))

In [7]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    """使用GPU计算模型在数据集上的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()
        if not device: # 如果没有指定device, 就用网络参数中的所给定的那个device。
            device = next(iter(net.parameters())).device
    
    metric = Accumulator(2)
    
    for X, y in data_iter:
        if isinstance(X, list):
            X = [x.to(device) for x in X]  # 将数据挪到device上。
        else:
            X = X.to(device)
            
        y = y.to(device)
        
        metric.add(accuracy(net(X), y), y.numel())
        
        return metric[0] / metric[1]

In [14]:
def train(net, train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    
    net.apply(init_weights)  # 对net里面的每一个parameter都去run一下init_weights这个函数。
    print("trainng on {}".format(device))
    
    net.to(device)
    
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    
    num_batches = len(train_iter)
    
    for epoch in range(num_epochs):
        metric = Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            
            X, y = X.to(device), y.to(device)
            
            y_hat = net(X)
            
            optimizer.zero_grad()
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            
            metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
            
            train_loss = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                print("epoch {}, train_loss {}, train_acc {}".format(epoch + (i + 1) / num_batches, 
                                                                  train_loss, train_acc))
    test_acc = evaluate_accuracy_gpu(net, test_iter)
    print(f'loss {train_loss:.3f}, train acc {train_acc:.3f}, 'f'test acc {test_acc:.3f}')

In [15]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [16]:
lr, num_epochs = 0.01, 10
train(net, train_iter, test_iter, num_epochs, lr, try_gpu())

trainng on cpu
epoch 0.19829424307036247, train_loss 2.288424643137122, train_acc 0.1353326612903226
epoch 0.39658848614072495, train_loss 2.032374075023077, train_acc 0.24718581989247312
epoch 0.5948827292110874, train_loss 1.7199561600189484, train_acc 0.3586749551971326
epoch 0.7931769722814499, train_loss 1.5144596995525463, train_acc 0.43550487231182794
epoch 0.9914712153518124, train_loss 1.3692845290707003, train_acc 0.4894489247311828
epoch 1.0, train_loss 1.3650486811319986, train_acc 0.49125
epoch 1.1982942430703625, train_loss 0.7270926576788708, train_acc 0.7240423387096774
epoch 1.396588486140725, train_loss 0.7130544057456396, train_acc 0.7295866935483871
epoch 1.5948827292110874, train_loss 0.6864783612203427, train_acc 0.7400033602150538
epoch 1.79317697228145, train_loss 0.6679184896651135, train_acc 0.7472908266129032
epoch 1.9914712153518124, train_loss 0.6565900720575805, train_acc 0.7520329301075269
epoch 2.0, train_loss 0.6562083961486816, train_acc 0.75225
epoch 

KeyboardInterrupt: 